In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle  # ✅ Added to save model as a pickle file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import os

# ✅ Enable Mixed Precision Training for Speed (GPU)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# ✅ Load Training & Testing Data
df_train = pd.read_csv("train.csv", dtype=str, low_memory=False)
df_test = pd.read_csv("test.csv", dtype=str, low_memory=False)

df_train['time'] = pd.to_datetime(df_train['time'], format="%H:%M:%S", errors="coerce")
df_test['time'] = pd.to_datetime(df_test['time'], format="%H:%M:%S", errors="coerce")

# ✅ Select Glucose-Related Columns (Ensuring Consistency)
bg_columns = [col for col in df_train.columns[1:] if "bg" in col and col in df_test.columns]
                                                                # ✅ Check if column exists in df_test

# ✅ Convert Glucose Data to Numeric
df_train[bg_columns] = df_train[bg_columns].apply(pd.to_numeric, errors="coerce")
df_test[bg_columns] = df_test[bg_columns].apply(pd.to_numeric, errors="coerce")

# ✅ Handle Missing Values (Forward & Backward Fill)
df_train[bg_columns] = df_train[bg_columns].ffill().bfill()
df_test[bg_columns] = df_test[bg_columns].ffill().bfill()

# ✅ Normalize Training & Testing Data (Avoiding Data Leakage)
scaler = MinMaxScaler()
scaler.fit(df_train[bg_columns])  # ✅ Fit only on training data

df_train[bg_columns] = scaler.transform(df_train[bg_columns])
df_test[bg_columns] = scaler.transform(df_test[bg_columns])

# ✅ Save Scaler for Future Use
with open("backend/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# ✅ Convert Data to Sequences (Time-Series Format for LSTM)
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)
        y.append(data.iloc[i+seq_length].values)
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X_train, y_train = create_sequences(df_train[bg_columns])
X_test, y_test = create_sequences(df_test[bg_columns])

# ✅ Define Optimized LSTM Model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(10, len(bg_columns))),
    BatchNormalization(),
    Dropout(0.2),

    LSTM(32, return_sequences=False),
    Dropout(0.2),

    Dense(len(bg_columns), dtype=tf.float32)  # ✅ Output is float32
])

optimizer = Adam(learning_rate=0.001)

# ✅ Compile Model outside the tf.function
model.compile(optimizer=optimizer, loss="mse") # Moved outside tf.function

# ✅ Train Model
model.fit(X_train, y_train, epochs=15, batch_size=128, validation_data=(X_test, y_test), verbose=1)

# ✅ Ensure backend directory exists
os.makedirs("backend", exist_ok=True)

# ✅ Save Model in H5 Format
model.save("backend/glucose_model.h5", save_format="h5")

# ✅ Save Model in Pickle Format
with open("backend/glucose_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Model Training Completed! Saved to 'backend/glucose_model.h5' and 'backend/glucose_model.pkl'")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 93s 63ms/step - loss: 0.0163 - val_loss: 0.0189
Epoch 2/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 150s 69ms/step - loss: 0.0025 - val_loss: 0.0187
Epoch 3/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 87s 63ms/step - loss: 0.0020 - val_loss: 0.0189
Epoch 4/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 141s 63ms/step - loss: 0.0019 - val_loss: 0.0187
Epoch 5/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 141s 62ms/step - loss: 0.0019 - val_loss: 0.0186
Epoch 6/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 143s 63ms/step - loss: 0.0019 - val_loss: 0.0186
Epoch 7/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 86s 62ms/step - loss: 0.0019 - val_loss: 0.0186
Epoch 8/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 87s 63ms/step - loss: 0.0018 - val_loss: 0.0187
Epoch 9/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 86s 62ms/step - loss: 0.0018 - val_loss: 0.0185
Epoch 10/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 89s 64ms/step - loss: 0.0018 - val_loss: 0.0184
Epoch 11/15
1383/1383 ━━━━━━━━━━━━━━━━━━━━ 140s 63ms/step - loss: 0.0018 - val_loss: 0.0184
Epo

✅ Model Training Completed! Saved to 'backend/glucose_model.h5' and 'backend/glucose_model.pkl'
